In [ ]:
#Import libraries
import pandas as pd
import numpy as np
#
import os
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau
from keras.utils import to_categorical
import cv2

In [ ]:
root_dir=os.path.abspath('/Users/savi/Documents/hackerearth/XRAY/5c5a5632-d-dl2_csvfiles/xray_final')
csv_path = os.path.join(root_dir , 'csvfiles/train.csv')
os.path.exists(csv_path)
images_dir = os.path.join(root_dir , 'train_/')
os.path.exists(images_dir)
testCsv_path = os.path.join(root_dir , 'csvfiles/test.csv')
testimages_dir = os.path.join(root_dir , 'test_/')

train = pd.read_csv(csv_path)
test = pd.read_csv(testCsv_path)

In [ ]:
import cv2
def read_img(img_path):
    img = cv2.imread(img_path)
    img = cv2.resize(img, (128,128))
    return img

from tqdm import tqdm
train_img = []
for img_path in tqdm(train['image_name'].values):
    train_img.append(read_img(os.path.join(images_dir , img_path)))

    
x_train = np.array(train_img, np.float32) / 255
mean_img = np.mean(x_train,axis=0)
std_img = np.std(x_train,axis=0)
x_train_norm = (x_train - mean_img) / std_img
class_list = train['detected'].tolist()
Y_train = {k:v+1 for v,k in enumerate(set(class_list))}
y_train = [Y_train[k] for k in class_list]


In [ ]:
# Set the CNN model 
# my CNN architechture is In -> [[Conv2D->relu]*2 -> MaxPool2D -> Dropout]*2 -> Flatten -> Dense -> Dropout -> Out

model0 = Sequential()

model0.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu', input_shape = (128,128,3)))
model0.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu'))
model0.add(MaxPool2D(pool_size=(2,2)))
model0.add(Dropout(0.25))


model0.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model0.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model0.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model0.add(Dropout(0.25))


model0.add(Flatten())
model0.add(Dense(256, activation = "relu"))
model0.add(Dropout(0.25))
model0.add(Dense(y_train.shape[1], activation = "softmax"))

optimizer = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)
model0.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])

learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)
epochs = 2

batch_size = 86
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)  # randomly flip images


datagen.fit(x_train_norm)

history=model0.fit_generator(datagen.flow(x_train_norm,Y_train, batch_size=batch_size, epochs=epochs, validation_split=0.2, callbacks=[learning_rate_reduction])


In [ ]:
#prediction
test_img = []
for img in tqdm(test['image_name'].values):
    test_img.append(read_img(os.path.join(testimages_dir , img))

x_test = np.array(test_img, np.float32) / 255
x_test_norm = (x_test - mean_img) / std_img

predictions = model0.predict(x_test_norm)
predictions = np.argmax(predictions, axis= 1)

y_maps = dict()
y_maps = {v:k for k, v in Y_train.items()}
pred_labels = [y_maps[k] for k in predictions]

sub = pd.DataFrame({'row_id':test.row_id, 'detected':pred_labels})
sub.to_csv(root_dir + 'submission.csv', index=False)
